In [1]:
import pandas as pd
import random
from pydataquery import DataQuery
from matplotlib.colors import LinearSegmentedColormap
from xbbg import blp
import numpy as np
import pytz
import time
from datetime import datetime, timedelta
import statsmodels.api as sm
from ipywidgets import interact, Dropdown, HBox, VBox, Button, Output, Text, widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
import re
import scipy.stats as stats
from adjustText import adjust_text
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import sympy as sp
import itertools
import warnings
import openpyxl
import subprocess
import time
import pyautogui
import pygetwindow as gw
import pyodbc
import ast
warnings.filterwarnings("ignore")

In [2]:
try:
    df_original = pd.read_parquet("Markit CDS.parquet/")
    
    to_date = datetime.today().date()
    from_date = df_original["close_date"].iloc[-1].date()
    
    from_date_str = from_date.strftime('%m/%d/%Y')
    to_date_str = to_date.strftime('%m/%d/%Y')
    
    conn_str = (
        f'DRIVER={{SQL Server}};'
        f'SERVER=BC-ODS-P1;'
        f'DATABASE=MarkitDB;'
        f'ApplicationIntent=ReadOnly;'
        f'Trusted_Connection=Yes;'
        f'Authentication=ActiveDirectoryIntegrated;'
    )
    
    conn = pyodbc.connect(conn_str)
    
    query1 = f"""
        DECLARE @FromDate DATE = '{from_date_str}';
        DECLARE @ToDate DATE = '{to_date_str}';
        
        SELECT sc.close_date, r.ticker, c.red, c.tier, c.docclause, c.ccy, sc.tenor, sc.spread
        FROM dbo.RedEntities r
        INNER JOIN dbo.MarkitCurves c ON r.red = c.red
        INNER JOIN dbo.MarkitSpreadCurve sc ON c.curve_id = sc.curve_id
        WHERE sc.close_date >= @FromDate AND sc.close_date <= @ToDate;
    """
    
    df_new = pd.read_sql(query1, conn)
    conn.close()

    df_old = df_original[df_original["close_date"]<pd.to_datetime(df_original["close_date"].iloc[-1])]

    df1 = pd.concat([df_old, df_new])
    df1 = df1.drop_duplicates()
    df1.to_parquet("Markit CDS.parquet")
except:
    df1 = pd.read_parquet("Markit CDS.parquet")
    hello = 1

In [7]:
markit_cds = list(set(df1["red"]))
markit_df1 = df1.copy()

res_codes = { "Full Restructuring": "CR14", "Modified Restructuring": "MR14",
          "Modified-Modified Restructurin": "MM14", "No Restructuring": "XR14"}

excel_df = None
all_dq = None
all_temp_cds = None

for rating_col in["SNRFIN"]:
# for rating_col in["IG","HY","EUR IG","EUR HY","SNRFIN"]:    
    dq = pd.read_excel("CDX Members.xlsx", sheet_name=rating_col)
    dq["Restructuring"] = dq["Restructuring"].apply(lambda x: res_codes[x])
    dq = dq[dq["Actual RED Code"].isin(markit_cds)].reset_index(drop=True).copy()
    dq.rename(columns={"Actual RED Code": "red"}, inplace=True)
    dq["Eqty"] = dq["Issuer Equity"]

    all_dq = pd.concat([all_dq, dq]).drop_duplicates(keep="first").reset_index(drop=True).copy()
    
    #################################
    df1 = markit_df1.copy()
    cds_df1 = df1[(df1["tier"]=="SNRFOR") & (df1["ccy"]=="USD") & (df1["tenor"]!="Spot") &\
        (df1["docclause"].str.endswith("14"))].drop(["tier","ccy"],axis=1).copy()
    
    #################################
    cds_df = cds_df1.copy()
    
    df = pd.merge(left=cds_df, right=dq, on="red", how="inner")
    df = df[df["docclause"]==df["Restructuring"]].drop(["RED Code","Restructuring"],axis=1).reset_index(drop=True).copy()
    cds_df = df[["close_date","ticker","red","tenor","spread","Eqty","Bond Name"]].copy()
    cds_df.rename(columns={"Eqty": "Eqty Name"}, inplace=True)
    
    cds_df["tenor"] = cds_df["tenor"].apply(lambda x: eval(x.replace("y","*1").replace("m","*(1/12)")))
    cds_df = cds_df[~pd.isna(cds_df["Eqty Name"])]
    cds_df = cds_df[~(cds_df["Eqty Name"].str.startswith("#"))].reset_index(drop=True).copy()

################################################################################################################################

    #############################################################
    issuers = list(sorted(set(cds_df["Eqty Name"])))
    last_dt = max(cds_df["close_date"])
    
    all_curves = None
    
    try:
        for issuer in issuers:
            df = cds_df[cds_df["Eqty Name"]==issuer]
            df = pd.pivot_table(df, values="spread", index="close_date", columns ="tenor")
            
            if not last_dt in df.index:
                df.loc[last_dt] = [np.nan] * len(df.columns)
            df = df.sort_index().ffill().copy()
            
            df[f"{issuer}_curve"] = [np.nan] * len(df)
        
            for idx in df.index:
                # idx = df.index[0]
                curve = df.loc[[idx],:].dropna(axis=1)
                curve = curve.iloc[:,:-1].copy()
                x = list((curve.columns))
                x2 = [item**2 for item in x]
                X = np.column_stack([x, x2])
                Y = list(curve.iloc[0])
        
                if len(X) > 2:
                    model = sm.OLS(Y, sm.add_constant(X)).fit()
                    df.loc[idx,f"{issuer}_curve"] = str([model.params[0], model.params[1], model.params[2]])
                    
                    # x_pred = list(np.linspace(min(x), max(x),100))
                    # x_pred2 = [item**2 for item in x_pred]
                    # X_pred = np.column_stack([x_pred, x_pred2])
                    # y_pred = model.predict(sm.add_constant(X_pred))
                    
                    # plt.plot(x, Y)
                    # plt.plot(x_pred, y_pred)
                    # title = f"{issuer} on {str(idx.date())}"
                    # plt.title(title)
                    # # plt.savefig(f"Curve Plots v2/{title}.png")
                    # plt.show()
                    # plt.close()
                    
                else:
                    df.loc[idx,f"{issuer}_curve"] = str([np.nan, np.nan, np.nan])
        
            all_curves = pd.concat([all_curves,df.iloc[:,[-1]]],axis=1)
        # all_curves.to_excel("All CDS Curves.xlsx")
    except:
        # all_curves = pd.read_excel("All CDS Curves.xlsx",index_col=0, parse_dates=True)
        hello=1
        
    all_curves1 = all_curves.copy()
    all_curves = all_curves.T.copy()
    all_curves.index = [item.split("_")[0] for item in all_curves.index]
    all_curves.index.name = "Eqty Name"
    all_curves.columns = [f'CDS_{item.date()}' for item in all_curves.columns]
    all_curves = all_curves.reset_index(drop=False).copy()
    temp_cds = all_curves.copy()
    all_temp_cds = pd.concat([all_temp_cds, temp_cds]).drop_duplicates(keep="first").reset_index(drop=True).copy()

################################################################################################################################
    # bql_list = []
    
    # fl = [f"""=BQL("filter(bonds(['""", f""" Equity']), payment_rank=='Sr Unsecured' AND crncy=='USD')", "id_isin, id_cusip, name, maturity, amt_outstanding")"""]
    # for item in issuers:
    #     bql_list += [(fl[0] + item + " " + fl[1])]
        
    # bql_list = [item.replace("\\", "") for item in bql_list]
    
    # workbook = openpyxl.load_workbook(r"J:\\HY Basis Data.xlsx")
    # sheet = workbook.active
    
    # for row in sheet.iter_rows():
    #     for cell in row:
    #         cell.value = None
    
    # start_col = 1
    # for item in bql_list:
    #     cell = sheet.cell(row=2, column=start_col)
    #     cell.value = item
    #     start_col += 6
    # workbook.save(r"J:\\HY Basis Data.xlsx")
    
    # file_path = r"J:\\HY Basis Data.xlsx"
    # window_title = "HY Basis Data - Excel"
    
    # subprocess.Popen(['start', 'excel', file_path], shell=True)
    # time.sleep(5)
    
    # excel_windows = [window for window in gw.getWindowsWithTitle('Excel')]
    
    # for window in excel_windows:
    #     if window_title in window.title:
    #         # time.sleep(0.5)
    #         window.activate()
    #         break
    
    # time.sleep(45)
    # pyautogui.hotkey('ctrl', 's')
    # time.sleep(1)
    # pyautogui.hotkey('alt', 'f4')
    
    # # for window in excel_windows:
    # #     if window_title in window.title:
    # #         # time.sleep(0.5)
    # #         window.activate()
    # #         break
    
    # # pyautogui.hotkey('alt', 'f4')
    # time.sleep(1.5)

################################################################################################################################

    issuer_dict = dict(zip(dq["Issuer Equity"],[item.split(" ")[0] for item in dq["Bond Name"]]))
    
    ############################################################################ Now get those bonds
    
    hy = pd.read_excel("All Bonds.xlsx",sheet_name=rating_col)
    all_df = None
    
    for i in range(len(hy.columns))[::6]:
        x = hy.iloc[:,i:i+6].dropna().copy()
        if len(x) > 0:
            x.columns = ['ID','ISIN', "CUSIP", 'Name', 'Maturity','Amt']
            x["Eqty Name"] = [issuer_dict[issuers[(int(i/6))]]] * len(x)
            x["Check Col"] = x.apply(lambda row: row["Name"].split(" ")[0]==row["Eqty Name"],axis=1)
            x = x[x["Check Col"]].drop("Check Col",axis=1)
            all_df = pd.concat([all_df, x])
    
    all_df['Time'] = round(((pd.to_datetime(all_df['Maturity'])-datetime.now()).dt.days/365),2)
    all_df = all_df[all_df["Time"]>=0]
    all_df = all_df[(all_df['Time']>=4) & (all_df['Time']<=10)]
    all_df = all_df[all_df['Amt']>=300*10**6]
    all_df = all_df.reset_index(drop=True)
    excel_df = pd.concat([excel_df, all_df])
all_df = excel_df.copy()

all_df = all_df.drop_duplicates(keep="first").reset_index(drop=True).copy()
################################################################################################################################

In [18]:
# set(markit_df1["tier"])
# markit_df1[(markit_df1["tier"]=="SUBLT2") & (markit_df1["ccy"]=="USD") &\
#     (markit_df1["close_date"]=="2025-08-26") & (markit_df1["tenor"]=="5y")]
set(markit_df1[markit_df1["red"]=="DD359M"]["tier"])

{'SNRFOR', 'SUBLT2'}

In [8]:
bql_list = []

fl = [f"""=BQL("filter(bonds(['""", f""" Equity']), payment_rank=='Sr Unsecured' AND crncy=='USD')", "id_isin, id_cusip, name, maturity, amt_outstanding")"""]
for item in issuers:
    bql_list += [(fl[0] + item + " " + fl[1])]
    
bql_list = [item.replace("\\", "") for item in bql_list]

workbook = openpyxl.load_workbook(r"J:\\HY Basis Data.xlsx")
sheet = workbook.active

for row in sheet.iter_rows():
    for cell in row:
        cell.value = None

start_col = 1
for item in bql_list:
    cell = sheet.cell(row=2, column=start_col)
    cell.value = item
    start_col += 6
workbook.save(r"J:\\HY Basis Data.xlsx")

file_path = r"J:\\HY Basis Data.xlsx"
window_title = "HY Basis Data - Excel"

subprocess.Popen(['start', 'excel', file_path], shell=True)
time.sleep(5)

excel_windows = [window for window in gw.getWindowsWithTitle('Excel')]

for window in excel_windows:
    if window_title in window.title:
        # time.sleep(0.5)
        window.activate()
        break

time.sleep(45)
pyautogui.hotkey('ctrl', 's')
# time.sleep(1)
# pyautogui.hotkey('alt', 'f4')

# time.sleep(1.5)


In [21]:
hy = pd.read_excel("All Bonds.xlsx",sheet_name=rating_col)
all_df = None

for i in range(len(hy.columns))[::6]:
    x = hy.iloc[:,i:i+6].dropna().copy()
    if len(x) > 0:
        display(issuer_dict[issuers[(int(i/6))]])
        x.columns = ['ID','ISIN', "CUSIP", 'Name', 'Maturity','Amt']
        x["Eqty Name"] = [issuer_dict[issuers[(int(i/6))]]] * len(x)
        display(x)
        x["Check Col"] = x.apply(lambda row: row["Name"].split(" ")[0]==row["Eqty Name"],axis=1)
        x = x[x["Check Col"]].drop("Check Col",axis=1)
        display(x)
        all_df = pd.concat([all_df, x])

all_df['Time'] = round(((pd.to_datetime(all_df['Maturity'])-datetime.now()).dt.days/365),2)
all_df = all_df[all_df["Time"]>=0]
all_df = all_df[(all_df['Time']>=4) & (all_df['Time']<=10)]
all_df = all_df[all_df['Amt']>=300*10**6]
all_df = all_df.reset_index(drop=True)

'CAR'

,ID,ISIN,CUSIP,Name,Maturity,Amt,Eqty Name
0,YO519655 Corp,US053773BK25,053773BK2,CAR 8 ⅜ 06/15/32,2032-06-15,600000000.0,CAR
1,BO490745 Corp,US053773BG13,053773BG1,CAR 4 ¾ 04/01/28,2028-04-01,500000000.0,CAR
2,YV709555 Corp,US053773BJ51,053773BJ5,CAR 8 ¼ 01/15/30,2030-01-15,700000000.0,CAR
3,AZ237070 Corp,US053773BC09,053773BC0,CAR 5 ¾ 07/15/27,2027-07-15,346668000.0,CAR
4,ZG179997 Corp,US053773BH95,053773BH9,CAR 8 02/15/31,2031-02-15,500000000.0,CAR
5,BO137083 Corp,US053773BF30,053773BF3,CAR 5 ⅜ 03/01/29,2029-03-01,600000000.0,CAR
6,BK769667 Corp,US053773BE64,053773BE6,CAR 5 ¾ 07/15/27,2027-07-15,303332000.0,CAR


,ID,ISIN,CUSIP,Name,Maturity,Amt,Eqty Name
0,YO519655 Corp,US053773BK25,053773BK2,CAR 8 ⅜ 06/15/32,2032-06-15,600000000.0,CAR
1,BO490745 Corp,US053773BG13,053773BG1,CAR 4 ¾ 04/01/28,2028-04-01,500000000.0,CAR
2,YV709555 Corp,US053773BJ51,053773BJ5,CAR 8 ¼ 01/15/30,2030-01-15,700000000.0,CAR
3,AZ237070 Corp,US053773BC09,053773BC0,CAR 5 ¾ 07/15/27,2027-07-15,346668000.0,CAR
4,ZG179997 Corp,US053773BH95,053773BH9,CAR 8 02/15/31,2031-02-15,500000000.0,CAR
5,BO137083 Corp,US053773BF30,053773BF3,CAR 5 ⅜ 03/01/29,2029-03-01,600000000.0,CAR
6,BK769667 Corp,US053773BE64,053773BE6,CAR 5 ¾ 07/15/27,2027-07-15,303332000.0,CAR


'WFRD'

,ID,ISIN,CUSIP,Name,Maturity,Amt,Eqty Name
0,BR928640 Corp,US947075AU14,947075AU1,WFRD 8 ⅝ 04/30/30,2030-04-30,1.600000e+09,WFRD


...

,ID,ISIN,CUSIP,Name,Maturity,Amt,Eqty Name
0,BN970577 Corp,US912909AU28,912909AU2,X 6 ⅞ 03/01/29,2029-03-01,474999000.0,X
1,EG460701 Corp,US912909AD03,912909AD0,X 6.65 06/01/37,2037-06-01,274142000.0,X
2,ZQ153470 Corp,US912909AT54,912909AT5,X 5 11/01/26,2026-11-01,373000.0,X


,ID,ISIN,CUSIP,Name,Maturity,Amt,Eqty Name
0,BN970577 Corp,US912909AU28,912909AU2,X 6 ⅞ 03/01/29,2029-03-01,474999000.0,X
1,EG460701 Corp,US912909AD03,912909AD0,X 6.65 06/01/37,2037-06-01,274142000.0,X
2,ZQ153470 Corp,US912909AT54,912909AT5,X 5 11/01/26,2026-11-01,373000.0,X


'YUM'

,ID,ISIN,CUSIP,Name,Maturity,Amt,Eqty Name
0,BV520889 Corp,US988498AR20,988498AR2,YUM 5 ⅜ 04/01/32,2032-04-01,1.000000e+09,YUM
1,ZO418868 Corp,US988498AN16,988498AN1,YUM 3 ⅝ 03/15/31,2031-03-15,1.050000e+09,YUM
2,BO612549 Corp,US988498AP63,988498AP6,YUM 4 ⅝ 01/31/32,2032-01-31,1.100000e+09,YUM
3,EG941813 Corp,US988498AD34,988498AD3,YUM 6 ⅞ 11/15/37,2037-11-15,3.250000e+08,YUM
4,ZR462627 Corp,US988498AL59,988498AL5,YUM 4 ¾ 01/15/30,2030-01-15,8.000000e+08,YUM
5,EJ899216 Corp,US988498AK76,988498AK7,YUM 5.35 11/01/43,2043-11-01,2.750000e+08,YUM


,ID,ISIN,CUSIP,Name,Maturity,Amt,Eqty Name
0,BV520889 Corp,US988498AR20,988498AR2,YUM 5 ⅜ 04/01/32,2032-04-01,1.000000e+09,YUM
1,ZO418868 Corp,US988498AN16,988498AN1,YUM 3 ⅝ 03/15/31,2031-03-15,1.050000e+09,YUM
2,BO612549 Corp,US988498AP63,988498AP6,YUM 4 ⅝ 01/31/32,2032-01-31,1.100000e+09,YUM
3,EG941813 Corp,US988498AD34,988498AD3,YUM 6 ⅞ 11/15/37,2037-11-15,3.250000e+08,YUM
4,ZR462627 Corp,US988498AL59,988498AL5,YUM 4 ¾ 01/15/30,2030-01-15,8.000000e+08,YUM
5,EJ899216 Corp,US988498AK76,988498AK7,YUM 5.35 11/01/43,2043-11-01,2.750000e+08,YUM


In [19]:
# issuer_dict = dict(zip(dq["Issuer Equity"],[item.split(" ")[0] for item in dq["Bond Name"]]))
hy = pd.read_excel(r"J:\\HY Basis Data.xlsx",skiprows=1)
hy.to_excel("Test1.xlsx")

In [4]:

###################################### 144A and REGS

blist = [f'/isin/{item}@BGN' for item in list(all_df["ISIN"])]
blist = blp.bdp(tickers=blist, flds=["144A_FLAG","IS_REG_S"])
blist.index = [item.rsplit("/",1)[1].split("@")[0] for item in blist.index]

blist.columns = ["144A","REGS"]
blist.index.name = "ISIN"
blist = blist.reset_index()
blist["REGS_144A"] = blist.apply(lambda row: f'{row["REGS"]}_{row["144A"]}',axis=1)
order = ["N_N", "Y_N", "Y_Y", "N_Y"]

all_df = pd.merge(left=all_df, right=blist, on="ISIN", how="outer")
all_df = all_df[[item for item in all_df.columns if not item in ["144A","REGS"]]]


df2 = all_df.copy()
df2["REGS_144A"] = pd.Categorical(df2["REGS_144A"], categories=order, ordered=True)
df2 = df2.sort_values(by="REGS_144A")
df2 = df2[~df2["Name"].duplicated(keep='first')].drop(["REGS_144A","Time"],axis=1).reset_index(drop=True).copy()
# df2.to_excel("HY Bonds List.xlsx")

##################################################################################

# df2 = pd.read_excel("HY Bonds List.xlsx", index_col=0, parse_dates=True)

t = [f"/isin/{item}@BGN" for item in list(df2["ISIN"])]
bbg = blp.bdh(tickers=t, flds="BLOOMBERG_MID_G_SPREAD", start_date=datetime.now()-timedelta(days=365*5))
bbg.to_parquet("Test1.parquet")
bbg = pd.read_parquet("Test1.parquet")

bbg1 = bbg.copy()
new = []
for item in bbg1.columns:
    new += ["BBG_" + item[0].replace("/isin/","").replace("@BGN","")]
bbg1.columns = new
bbg1.index = pd.to_datetime(bbg1.index)

########################### choose bbg or dq sprds
bbg1.columns = [item.split("_")[1] for item in bbg1.columns]
bbg1.index = [f"Sprd_{str(item.date())}" for item in bbg1.index]
bbg1 = bbg1.T.copy()
bbg1.index.name = "ISIN"
bbg1 = bbg1.reset_index(drop=False).copy()

In [5]:
df3 = df2.copy()
for dt in pd.to_datetime(bbg.index):
    df3[f'Mat_Time_{dt.date()}'] = [((pd.to_datetime(item) - dt).days/365) for item in df3["Maturity"]]

red2 = dict(zip([item.split(" ")[0] for item in all_dq["Bond Name"]], all_dq["Eqty"]))
df3["Eqty Name"] = df3["Eqty Name"].apply(lambda x: red2[x])

######################################################################

df4 = pd.merge(left = df3, right = all_temp_cds, on="Eqty Name", how="outer")
df4 = df4[~pd.isna(df4["ISIN"])].reset_index(drop=True).copy()
dt_list = [item.replace("CDS_","") for item in df4.columns if item.startswith("CDS_")]

df4A = df4.copy()

for dt in dt_list:
    if f"Mat_Time_{dt}" in df4A.columns and f"CDS_{dt}" in df4A.columns:
        df4A[f'Mat_Matched_CDS_{dt}'] = df4A[f'CDS_{dt}'].apply(lambda x: ast.\
            literal_eval(x)[0] if isinstance(x,str) else np.nan) +\
        df4A[f'Mat_Time_{dt}'].apply(lambda x: x) *\
        df4A[f'CDS_{dt}'].apply(lambda x: ast.literal_eval(x)[1] if isinstance(x,str) else np.nan) +\
        df4A[f'Mat_Time_{dt}'].apply(lambda x: x**2) *\
        df4A[f'CDS_{dt}'].apply(lambda x: ast.literal_eval(x)[2] if isinstance(x,str) else np.nan)
      
df4A = df4A[[col for col in df4A.columns if not "_" in col] +\
    [col for col in df4A.columns if col.startswith("Mat_Matched_CDS_")]]

In [6]:
df5 = pd.merge(left=df4A, right=bbg1, how='outer', on="ISIN")
last_update = max(dt_list)
for dt in dt_list:
    try:
        df5[f"Basis_{dt}"] = df5[f"Mat_Matched_CDS_{dt}"] - df5[f"Sprd_{dt}"]
    except:
        hello = 1
l1 = list(set([col for col in df5.columns if f'{last_update}' in col] + [col for col in df5.columns if 'Basis' in col] ))
l2 = [item for item in l1 if f'{last_update}' in item] + [item for item in l1 if not f'{last_update}' in item]
df5 = df5[[col for col in df5.columns if not "_" in col] + l2]
df5 = df5[~pd.isna(df5[f"Sprd_{last_update}"])].reset_index(drop=True).copy()

red = all_dq[["Eqty","red"]].dropna().copy()
# red_eqty_dict1 = dict(zip([item.split(" ")[0] for item in list(red["Eqty"])], list(red["red"])))
red_eqty_dict1 = dict(zip(list(red["Eqty"]), list(red["red"])))

df5["CDS RED Code"] = df5["Eqty Name"].apply(lambda x: red_eqty_dict1[x])

####################################### Zscore and 1Y High/Low Calc

dfz = df5[["ISIN"] + [item for item in df5.columns if "Basis" in item]]
dfz = dfz.set_index("ISIN").T
dfz.index = [pd.to_datetime(item.split("_")[1]).date() for item in dfz.index]
dfz = dfz.sort_index()

all_z_df = None
for period in [3*22, 6*22, 12*22]:
    x = ((dfz-dfz.rolling(period, min_periods = int(0.6*period)).mean())/dfz.rolling(period,\
                  min_periods = int(0.6*period)).std()).iloc[[-1],:].copy()
    x.index = [f'ZScore_{int(period/22)}M']
    all_z_df = pd.concat([all_z_df , x])

max_1y = pd.DataFrame(dfz.iloc[-12*22:,:].max()).T
max_1y.index = [f'Basis 1Y High']
min_1y = pd.DataFrame(dfz.iloc[-12*22:,:].min()).T
min_1y.index = [f'Basis 1Y Low']

all_z_df = pd.concat([all_z_df , max_1y, min_1y])
all_z_df = all_z_df.T
all_z_df = all_z_df.reset_index()

df6 = pd.merge(left=df5[[col for col in df5.columns if not col.startswith("Basis_") or dt in col]],
               right=all_z_df, on="ISIN", how="outer")
df6 = df6[["Name", "ISIN", "Eqty Name", "CDS RED Code", f"Basis_{last_update}",
           f"Sprd_{last_update}", f"Mat_Matched_CDS_{last_update}"] + list(df6.columns)[-5:]]


################################### Add PX
px_list = [f"/isin/{i}@BGN" for i in list(df6["ISIN"])]
px = blp.bdp(tickers=px_list, flds="px_last")
px.to_parquet("Test2.parquet")
px = pd.read_parquet("Test2.parquet")

px1 = px.copy()
px1.index = [item.replace("/isin/","").replace("@BGN","") for item in px1.index]
px1.columns = ["Price"]
px1.index.name ="ISIN"
px1 = px1.reset_index().copy()
df6 = pd.merge(left=df6, right=px1, how="outer", on="ISIN")


################################################## Adding IG/HY
blist = [f"/isin/{item}@BGN" for item in list(df6["ISIN"])]
ig_hy = blp.bdp(tickers=blist, flds="BB_COMPSTE_RATING_IG_HY_INDCTR")
ig_hy.to_parquet("Test3.parquet")
ig_hy = pd.read_parquet("Test3.parquet")

ig_hy["ISIN"] = [item.replace("/isin/","").replace("@BGN","") for item in ig_hy.index]
ig_hy.columns = ["IG_HY","ISIN"]
ig_hy = ig_hy.reset_index(drop=True).copy()
df6 = pd.merge(left=df6, right=ig_hy, how="outer", on="ISIN")
df6 = df6.sort_values(by="Eqty Name").reset_index(drop=True).copy()

################################################## 

df7 = df6.copy()
df7["Time"] = df7["Name"].apply(lambda x: x.split("/")[-1])
df7 = df7.sort_values(by=["Eqty Name","Time"],ascending=True).\
    reset_index(drop=True).drop(["Eqty Name","Time","ISIN"],axis=1).copy()
df7 = df7.set_index("Name")
df7.columns = df7.columns.str.replace(f'_{last_update}',"")
df7.index.name = f'{pd.to_datetime(last_update).strftime("%d-%b")}'
df7 = round(df7,2)
df7 = df7[["IG_HY","CDS RED Code","Price","Sprd","Mat_Matched_CDS","Basis",
           "Basis 1Y Low","Basis 1Y High","ZScore_3M","ZScore_6M","ZScore_12M"]]

df7["Name"] = [item.split(" ")[0] for item in df7.index]
df7["Year"] = [eval(item.split("/")[-1]) for item in df7.index]
df7 = df7.sort_values(by=["Name","Year"], ascending=True).drop(["Name","Year"],axis=1).copy()

px_new = blp.bdh(tickers=blist, flds="PX_LAST", start_date = datetime.now()-timedelta(days=5*365))
px_new.columns = [item.replace("/isin/","").replace("@BGN","") + "_Price" for item in blist]
px_new.to_parquet("Test4.parquet")
px_new = pd.read_parquet("Test4.parquet")

In [7]:

# dq_bonds = ['US00130HCC79','US00130HCG83','US00206RCP55','US00206RMM15','US012873AH83','US012873AK13','US013092AE14','US01309QAB41','US023135AP19','US023551AF16','US023551AJ38','US023551AM66','US026874DC84','US031162DQ06','US03674XAS53','US03743QAF54','US03743QAQ10','US05368VAA44','US05368VAB27','US058498AW66','US058498AX40','US058498BA38','US097023AU94','US097023CJ22','US097023CN34','US097023CP81','US097023CY98','US097023DC69','US097023DR39','US097023DS12','US11135FAS02','US11135FBD24','US11135FBF71','US11135FBH38','US11135FBK66','US11135FBL40','US11135FBT75','US126650DJ69','US126650DU15','US126650ED80','US12769GAA85','US12769GAD25','US134429BJ73','US149123CH22','US165167DH73','US166756AS52','US185899AL57','US185899AN14','US185899AP61','US185899AQ45','US185899AR28','US185899AS01','US23331ABT51','US23918KAS78','US23918KAT51','US23918KAW80','US23918KAY47','US247361A329','US247361ZT81','US251799AA02','US25179MBF95','US25179SAD27','US254709AS70','US26884LAG41','US26884LAN91','US26884LAR06','US26884LBA61','US26884LBC28','US292505AD65','US30212PAR64','US30212PBH73','US30231GBN16','US337932AL12','US337932AP26','US345370CA64','US345370CX67','US345370DA55','US345370DB39','US35671DBJ37','US35671DCD57','US35671DCF06','US35671DCH61','US364760AP35','US364760AQ18','US36962GXZ26','US370425RZ53','US37045VAH33','US37045VAY65','US37045VAZ31','US382550BJ95','US382550BK68','US382550BR12','US382550BS94','US404119CA57','US404119CC14','US404119CK30','US404119CQ00','US404119CT49','US404119DB22','US404121AK12','US437076CB65','US44106MAY84','US44106MBB72','US458140BR09','US46284VAF85','US46284VAJ08','US46284VAL53','US46284VAN10','US46284VAQ41','US48666KAY55','US48666KAZ21','US48666KBA60','US500255AX28','US501797AW48','US513272AD65','US513272AE49','US526057CY87','US55262CAJ99','US552676AT59','US552676AU23','US552953CJ87','US552953CK50','US55616XAM92','US55617LAR33','US55617LAS16','US571903BE27','US571903BF91','US571903BG74','US571903BH57','US626717AP72','US629377CR16','US629377CS98','US629377CW01','US629377CX83','US62957HAP01','US62957HAQ83','US63861CAD11','US63861CAE93','US63861CAF68','US63938CAN83','US63938CAP32','US63938CAQ15','US64110LAU08','US64110LAV80','US651229BD74','US651229BE57','US651229BF23','US674599DD43','US674599DE26','US674599EA94','US674599ED34','US674599EF81','US674599EK76','US674599EL59','US677347CH71','US680665AK27','US680665AN65','US68389XBV64','US68389XCE31','US68389XCH61','US68389XCJ28','US68622FAB76','US68622TAB70','US69047QAC69','US698900AG20','US737446AP91','US737446AQ74','US737446AR57','US737446AV69','US737446AX26','US745867AM30','US745867AP60','US745867AT82','US75513ECR09','US780153BU54','US780153BV38','US780153BW11','US78442FAZ18','US785931AA40','US785931AB23','US78646UAA79','US78646UAB52','US81211KAK60','US812127AB45','US812127AC28','US81761LAE20','US845467AS85','US845467AT68','US87264ABF12','US87264ABT16','US87264ABW45','US87264ABX28','US87264ACB98','US87264ACQ67','US87264ACV52','US87264ADT97','US87612GAA94','US87901JAH86','US88033GAV23','US88947EAU47','US893830AF64','US893830BZ10','US902494AZ66','US90353TAK60','US90353TAN00','US90353TAP57','US925524AH30','US925524AV24','US92556HAB33','US92556HAD98','US963320AY28','US963320AZ92','US963320BA33','US963320BC98','US963320BD71','US963320BE54','US988498AL59','US988498AN16','US988498AP63','US988498AR20']

dq_bonds = list(df5["ISIN"])
all_labels = dict(zip(df5["ISIN"],[f"DB(CREDIT,HY,BOND,{item},MDUR)" for item in df5["CUSIP"]]))

labels = {}
for item in dq_bonds:
    labels[f"{item}_Dur"] = all_labels[item]

try:
    # df1 = pd.read_excel("DQ HY Duration Data.xlsx",index_col=0, parse_dates=True)
    dq = DataQuery(
        client_id='jbAIMF2Tkp0JO3sc',
        client_secret='d7qfzgt55pddjs352sgxosFyI4t2eja07k7opbi6wg9oqjc1OjkdAksn1btmnugeMjchcx2vwTsJupw',
        calendar = 'CAL_USBANK',
    )
    
    job = dq.create_job(expressions = list(labels.values()))
    dq.start_date = str((datetime.now()-timedelta(days=5*365)).date())
    job.execute()
    df = job.to_pivot_table()
    df = df.T
    df.index.name = 'Date'
    df.index = pd.to_datetime(df.index, format='%Y%m%d').date
    df.columns.name = None
    
    df1 = pd.DataFrame()
    for key in labels:
        df1[key] = df[labels[key]]
    
    df1 = df1[list(labels.keys())].copy()
    clear_output(wait=False)
    df1.dropna(axis=1, how='all', inplace=True)
    df1.to_excel("DQ HY Duration Data.xlsx")
except:
    df1 = pd.read_excel("DQ HY Duration Data.xlsx",index_col=0, parse_dates=True)

In [8]:
A = df4A[["ISIN"]+[col for col in df4A.columns if col.startswith("Mat_Matched_CDS_")]].set_index("ISIN")
A = A.T
A.index = pd.to_datetime(A.index.str.replace("Mat_Matched_CDS_",""))
A.columns = [f"{item}_Mat_Matched_CDS_" for item in A.columns]

B = df5[["ISIN"]+[col for col in df5.columns if col.startswith("Basis_")]].set_index("ISIN")
B = B.T
B.index = pd.to_datetime(B.index.str.replace("Basis_",""))
B.columns = [f"{item}_Basis" for item in B.columns]

C = bbg1[["ISIN"]+[col for col in bbg1.columns if col.startswith("Sprd_")]].set_index("ISIN")
C = C.T
C.index = pd.to_datetime(C.index.str.replace("Sprd_",""))
C.columns = [f"{item}_G-Sprd" for item in C.columns]

D = px_new.copy()

E = df1.copy()
# E = E.set_index("close_date")

for item in ["A","B","C","D","E"]:
    globals()[f"{item}"].index = pd.to_datetime(globals()[f"{item}"].index)

F = pd.concat([A,B,C,D,E], axis=1).sort_index().copy()

In [9]:
# ######################### HY bonds TR data from BBG using BQL

# import openpyxl
# import subprocess
# import time
# import pyautogui
# import pygetwindow as gw
    
# if True:
#     bql_list = []
    
#     fl = [f'=BQL("', f"""ISIN", "return_series(calc_interval=range(-5y,0d,frq=d))")"""]
    
#     issuers = df6["ISIN"].to_list()
    
#     for item in issuers:
#         bql_list += [(fl[0] + item + " " + fl[1])]
        
#     bql_list = [item.replace("\\", "") for item in bql_list]
    
#     workbook = openpyxl.load_workbook(r"J:\\55 BQL HY Bonds Rtn Data.xlsx")
#     sheet = workbook.active
    
#     for row in sheet.iter_rows():
#         for cell in row:
#             cell.value = None
    
#     start_col = 1
#     for item in bql_list:
#         cell = sheet.cell(row=2, column=start_col)
#         cell.value = item
#         start_col += 2
#     workbook.save(r"J:\\55 BQL HY Bonds Rtn Data.xlsx")
    
#     file_path = r"J:\\55 BQL HY Bonds Rtn Data.xlsx"
#     window_title = "55 BQL HY Bonds Rtn Data - Excel"
    
#     subprocess.Popen(['start', 'excel', file_path], shell=True)
#     time.sleep(40)
    
#     excel_windows = [window for window in gw.getWindowsWithTitle('Excel')]
    
#     for check in range(2):
#         for window in excel_windows:
#             if window_title in window.title:
#                 # time.sleep(0.25)
#                 window.activate()
#                 pyautogui.hotkey('ctrl', 's')
#                 time.sleep(1)
#                 pyautogui.hotkey('alt', 'f4')
#                 # break
    
# time.sleep(0.5)

# hy = pd.read_excel(r"J:\\55 BQL HY Bonds Rtn Data.xlsx",skiprows=1,parse_dates=True)
# hy = hy.iloc[1:,:]
# hy = hy[[hy.columns[0]] + [col for col in hy.columns if "ISIN" in col]]
# hy.columns = ["Date"] + [item.replace(" ISIN","_Daily_Rtn") for item in list(hy.columns)[1:]]
# hy = hy.set_index("Date")
# hy.index = pd.to_datetime(hy.index)
# hy = (1+hy).cumprod()
# hy_col = hy.columns
# for col in hy_col:
#     hy[f'{col.split("_")[0]} FWD_1M_Rtn'] = (hy[col].shift(-21) / hy[col] - 1) * 100
#     hy[f'{col.split("_")[0]} FWD_3M_Rtn'] = (hy[col].shift(-63) / hy[col] - 1) * 100
#     hy[f'{col.split("_")[0]} FWD_6M_Rtn'] = (hy[col].shift(-126) / hy[col] - 1) * 100
# hy = hy[[col for col in hy.columns if "FWD" in col]]

In [10]:
# df = pd.concat([F, hy],axis=1).sort_index().copy()

df = F.sort_index().copy()
df.columns = df.columns.str.replace("_Mat_Matched_CDS_"," Mat. Matched CDS").\
    str.replace("_Basis"," Basis").str.replace("_Dur"," Duration").\
    str.replace("_Price"," Price").str.replace("_G-Sprd"," G-Sprd")

bond_dict = dict(zip(list(df2["ISIN"]),list(df2["Name"])))
df.columns = [f"{bond_dict[item.split(" ",1)[0]]} {item.split(" ",1)[1]}"  for item in df.columns]
df = df.dropna(how="all").copy()
df.columns = [f"{item.split("/",1)[0].rsplit(" ",1)[0]}% due {item.\
    rsplit("/",1)[1]}" for item in df.columns]

In [11]:
tickers_bbg = ['IBOXHY INDEX', 'IBOXIG INDEX']
fields_bbg = ['PX_LAST', 'CONTRBTD_ZSPREAD', 'OAS_SOVEREIGN_CURVE', 'OPTION_ADJ_DURATION_SOV_CRV', 'YIELD_TO_MATURITY']
start_date_bbg = "2000-01-01"
end_date_bbg = datetime.now().strftime('%Y-%m-%d')
df_bbg = blp.bdh(tickers=tickers_bbg, flds=fields_bbg, start_date=start_date_bbg, end_date=end_date_bbg)
list1_bbg = ['TR', 'Z-Sprd', 'OAS', 'Dur', 'Yield']
df_bbg.columns = ['BBG iBoxx HY ' + item for item in list1_bbg] + ['BBG iBoxx IG ' + item for item in list1_bbg] 
df_bbg.drop(['BBG iBoxx HY Dur', 'BBG iBoxx IG Yield'], axis=1, inplace=True) # First is wrong and second incomplete
df_bbg = df_bbg.astype(float)

tick = ['USGG5YR INDEX','USGG7YR INDEX','USGG10YR INDEX']
all_x = None
x = blp.bdh(tickers = tick, flds = 'PX_LAST', start_date = df_bbg.index[0])#, end_date = df_bbg.index[-1])
for col in ['BBG']:# + list(set([item.split(' ',1)[0] for item in df_back.columns])):
    x1 = x.copy()
    x1.columns = [f'{col} US 5Y', f'{col} US 7Y', f'{col} US 10Y']
    all_x = pd.concat([all_x,x1],axis=1)
df_bbg = pd.concat([df_bbg,all_x],axis=1)

tick = ['COA Comdty','VIX Index']
x = blp.bdh(tickers = tick, flds='PX_LAST', start_date = df_bbg.index[0])#, end_date = df_bbg.index[-1])
x.columns = ['BBG COA Comdty','BBG VIX Index']
x = x.sort_index()
df_bbg = pd.concat([df_bbg,x],axis=1)
df_bbg.index = pd.to_datetime(df_bbg.index)

backup = df.copy()
df = pd.concat([backup, df_bbg], axis=1).sort_index().copy()

In [12]:
options = sorted(backup.columns, key=lambda item: (item.split(' ', 1)[0], 2000 + \
            eval(item.split('due ', 1)[1].split(' ', 1)[0]) if eval(item.split('due ', 1)[1].split(' ', 1)[0])\
                    < 2000 else eval(item.split('due ', 1)[1].split(' ', 1)[0])))

options += list(df_bbg.columns)
fwd = [item for item in options if 'FWD_' in item]
non_fwd = [item for item in options if not('FWD_' in item)]
options = non_fwd + fwd

time = ['All','2Y','1Y','6M','3M']

#################################### Changing order of names
o1 = [item for item in options if ((not 'FWD' in item) and ('%' in item))]
o2 = [item for item in options if not item in o1]

desired_order = ["Price", "G-Sprd", "Basis", "Duration", "CDS"]

def reorder_items_in_chunks(items, order):
    chunk_size = len(set([item.rsplit(' ',1)[1] for item in options if ((not 'FWD' in item) and ('%' in item))]))
    reordered = []
    for i in range(0, len(items), chunk_size):
        chunk = items[i:i + chunk_size]
        valid_items = [item for item in chunk if len(item.rsplit(' ', 1)) == 2]
        reordered_chunk = sorted(valid_items, key=lambda x: order.index(x.rsplit(' ', 1)[1]))
        reordered.extend(reordered_chunk)
    return reordered

o1_reordered = reorder_items_in_chunks(o1, desired_order)
options = o1 + o2

In [13]:
############ Default bonds
# x = df6[["Name","ISIN","Eqty Name"]].copy()
# issue = blp.bdp(tickers=[f"/isin/{item}@BGN" for item in x["ISIN"]], flds="ISSUE_DT")
# issue1 = issue.copy()
# issue1.index = issue1.index.str.replace("/isin/","").str.replace("@BGN","")
# issue1.index.name = "ISIN"
# issue1 = issue1.reset_index()
# x = pd.merge(left=x, right=issue1, on="ISIN", how="outer")
# x["Age"] = round((datetime.now() - pd.to_datetime(x["issue_dt"])).dt.days/365,2)
# x = x[["Name","Eqty Name","Age"]].copy()
# x = x.sort_values(by=["Eqty Name","Age"], ascending=[True, False]).reset_index(drop=True).copy()
# x = x.drop_duplicates(subset="Eqty Name", keep="first")[["Name"]].copy()
# x["Name"] = x["Name"].apply(lambda x: x.rsplit(" ",1)[0] + "% due " + x.rsplit("/",1)[-1])
# x["MMC"] = x["Name"].apply(lambda x: [f"{x.split(" ",1)[1]} Mat. Matched CDS", f"{x.split(" ",1)[1]} Basis"])
# x["Name"] = x["Name"].apply(lambda x: f"{x.split(" ",1)[0]}")
# dict(zip(x["Name"], x["MMC"]))



default_values_dict = \
{'TOL': ['3.8% due 29 Mat. Matched CDS', '3.8% due 29 Basis'],
 'ZIGGO': ['5 ⅛% due 30 Mat. Matched CDS', '5 ⅛% due 30 Basis'],
 'CAR': ['8% due 31 Mat. Matched CDS', '8% due 31 Basis'],
 'WFRD': ['8 ⅝% due 30 Mat. Matched CDS', '8 ⅝% due 30 Basis'],
 'MPW': ['3 ½% due 31 Mat. Matched CDS', '3 ½% due 31 Basis'],
 'OI': ['4 ¾% due 30 Mat. Matched CDS', '4 ¾% due 30 Basis'],
 'CHTR': ['4 ¾% due 30 Mat. Matched CDS', '4 ¾% due 30 Basis'],
 'HOUS': ['5 ¼% due 30 Mat. Matched CDS', '5 ¼% due 30 Basis'],
 'SIRI': ['4 ⅛% due 30 Mat. Matched CDS', '4 ⅛% due 30 Basis'],
 'TELEFO': ['8 ¼% due 30 Mat. Matched CDS', '8 ¼% due 30 Basis'],
 'SIEGR': ['2.15% due 31 Mat. Matched CDS', '2.15% due 31 Basis'],
 'CSCHLD': ['5 ¾% due 30 Mat. Matched CDS', '5 ¾% due 30 Basis'],
 'SPG': ['2.45% due 29 Mat. Matched CDS', '2.45% due 29 Basis'],
 'RGCARE': ['10% due 32 Mat. Matched CDS', '10% due 32 Basis'],
 'TEVA': ['8 ⅛% due 31 Mat. Matched CDS', '8 ⅛% due 31 Basis'],
 'HLT': ['4 ⅞% due 30 Mat. Matched CDS', '4 ⅞% due 30 Basis'],
 'VST': ['7 ¾% due 31 Mat. Matched CDS', '7 ¾% due 31 Basis'],
 'ALIANT': ['5 ⅞% due 29 Mat. Matched CDS', '5 ⅞% due 29 Basis'],
 'AXL': ['5% due 29 Mat. Matched CDS', '5% due 29 Basis'],
 'IMBLN': ['5 ½% due 30 Mat. Matched CDS', '5 ½% due 30 Basis'],
 'DGELN': ['2 ⅜% due 29 Mat. Matched CDS', '2 ⅜% due 29 Basis'],
 'TMUS': ['2 ⅞% due 31 Mat. Matched CDS', '2 ⅞% due 31 Basis'],
 'URI': ['5 ¼% due 30 Mat. Matched CDS', '5 ¼% due 30 Basis'],
 'DOW': ['7 ⅜% due 29 Mat. Matched CDS', '7 ⅜% due 29 Basis'],
 'M': ['4 ½% due 34 Mat. Matched CDS', '4 ½% due 34 Basis'],
 'JCI': ['1 ¾% due 30 Mat. Matched CDS', '1 ¾% due 30 Basis'],
 'AALLN': ['2 ⅞% due 31 Mat. Matched CDS', '2 ⅞% due 31 Basis'],
 'OGN': ['5 ⅛% due 31 Mat. Matched CDS', '5 ⅛% due 31 Basis'],
 'MEDIND': ['5 ¼% due 29 Mat. Matched CDS', '5 ¼% due 29 Basis'],
 'TTEFP': ['5.15% due 34 Mat. Matched CDS', '5.15% due 34 Basis'],
 'PRUFIN': ['3 ⅛% due 30 Mat. Matched CDS', '3 ⅛% due 30 Basis'],
 'NRUC': ['8% due 32 Mat. Matched CDS', '8% due 32 Basis'],
 'LINTA': ['4% due 29 Mat. Matched CDS', '4% due 29 Basis'],
 'RIG': ['7 ½% due 31 Mat. Matched CDS', '7 ½% due 31 Basis'],
 'HILCRP': ['6% due 31 Mat. Matched CDS', '6% due 31 Basis'],
 'CE': ['6.629% due 32 Mat. Matched CDS', '6.629% due 32 Basis'],
 'HST': ['3 ⅜% due 29 Mat. Matched CDS', '3 ⅜% due 29 Basis'],
 'TRPCN': ['5.6% due 34 Mat. Matched CDS', '5.6% due 34 Basis'],
 'BRITEL': ['9 ⅝% due 30 Mat. Matched CDS', '9 ⅝% due 30 Basis'],
 'RDSALN': ['2 ⅜% due 29 Mat. Matched CDS', '2 ⅜% due 29 Basis'],
 'BMCAUS': ['4 ⅜% due 30 Mat. Matched CDS', '4 ⅜% due 30 Basis'],
 'NEE': ['2 ¾% due 29 Mat. Matched CDS', '2 ¾% due 29 Basis'],
 'HCA': ['3 ½% due 30 Mat. Matched CDS', '3 ½% due 30 Basis'],
 'NESNVX': ['1 ¼% due 30 Mat. Matched CDS', '1 ¼% due 30 Basis'],
 'AIFP': ['2 ¼% due 29 Mat. Matched CDS', '2 ¼% due 29 Basis'],
 'AEP': ['2.3% due 30 Mat. Matched CDS', '2.3% due 30 Basis'],
 'AES': ['3.95% due 30 Mat. Matched CDS', '3.95% due 30 Basis'],
 'OMF': ['5 ⅜% due 29 Mat. Matched CDS', '5 ⅜% due 29 Basis'],
 'AIG': ['3 ⅞% due 35 Mat. Matched CDS', '3 ⅞% due 35 Basis'],
 'ALL': ['5.35% due 33 Mat. Matched CDS', '5.35% due 33 Basis'],
 'ALLY': ['8% due 31 Mat. Matched CDS', '8% due 31 Basis'],
 'AMGN': ['2.45% due 30 Mat. Matched CDS', '2.45% due 30 Basis'],
 'AMZN': ['4.8% due 34 Mat. Matched CDS', '4.8% due 34 Basis'],
 'APA': ['4 ¼% due 30 Mat. Matched CDS', '4 ¼% due 30 Basis'],
 'ARW': ['2.95% due 32 Mat. Matched CDS', '2.95% due 32 Basis'],
 'AVGO': ['5% due 30 Mat. Matched CDS', '5% due 30 Basis'],
 'AVT': ['3% due 31 Mat. Matched CDS', '3% due 31 Basis'],
 'AXP': ['4.42% due 33 Mat. Matched CDS', '4.42% due 33 Basis'],
 'AZN': ['1 ⅜% due 30 Mat. Matched CDS', '1 ⅜% due 30 Basis'],
 'AZO': ['4% due 30 Mat. Matched CDS', '4% due 30 Basis'],
 'BA': ['6 ⅛% due 33 Mat. Matched CDS', '6 ⅛% due 33 Basis'],
 'BALN': ['3.4% due 30 Mat. Matched CDS', '3.4% due 30 Basis'],
 'BACR': ['2.645% due 31 Mat. Matched CDS', '2.645% due 31 Basis'],
 'BAX': ['3.95% due 30 Mat. Matched CDS', '3.95% due 30 Basis'],
 'BBY': ['1.95% due 30 Mat. Matched CDS', '1.95% due 30 Basis'],
 'BMY': ['1.45% due 30 Mat. Matched CDS', '1.45% due 30 Basis'],
 'BRK': ['6 ½% due 34 Mat. Matched CDS', '6 ½% due 34 Basis'],
 'BSX': ['2.65% due 30 Mat. Matched CDS', '2.65% due 30 Basis'],
 'CAG': ['8 ¼% due 30 Mat. Matched CDS', '8 ¼% due 30 Basis'],
 'CAH': ['5.45% due 34 Mat. Matched CDS', '5.45% due 34 Basis'],
 'CMCSA': ['7.05% due 33 Mat. Matched CDS', '7.05% due 33 Basis'],
 'CNQCN': ['7.2% due 32 Mat. Matched CDS', '7.2% due 32 Basis'],
 'COF': ['2.7% due 30 Mat. Matched CDS', '2.7% due 30 Basis'],
 'COXENT': ['4.8% due 35 Mat. Matched CDS', '4.8% due 35 Basis'],
 'CPB': ['2 ⅜% due 30 Mat. Matched CDS', '2 ⅜% due 30 Basis'],
 'CSCO': ['4.95% due 31 Mat. Matched CDS', '4.95% due 31 Basis'],
 'CSX': ['2.4% due 30 Mat. Matched CDS', '2.4% due 30 Basis'],
 'CVS': ['4 ⅞% due 35 Mat. Matched CDS', '4 ⅞% due 35 Basis'],
 'D': ['6.3% due 33 Mat. Matched CDS', '6.3% due 33 Basis'],
 'DAL': ['3 ¾% due 29 Mat. Matched CDS', '3 ¾% due 29 Basis'],
 'DE': ['7 ⅛% due 31 Mat. Matched CDS', '7 ⅛% due 31 Basis'],
 'DGX': ['2.95% due 30 Mat. Matched CDS', '2.95% due 30 Basis'],
 'DHI': ['5% due 34 Mat. Matched CDS', '5% due 34 Basis'],
 'DHR': ['2.6% due 29 Mat. Matched CDS', '2.6% due 29 Basis'],
 'DIS': ['7% due 32 Mat. Matched CDS', '7% due 32 Basis'],
 'DRI': ['6.3% due 33 Mat. Matched CDS', '6.3% due 33 Basis'],
 'DT': ['8 ¼% due 30 Mat. Matched CDS', '8 ¼% due 30 Basis'],
 'DVN': ['7 ⅞% due 31 Mat. Matched CDS', '7 ⅞% due 31 Basis'],
 'EDF': ['6 ¼% due 33 Mat. Matched CDS', '6 ¼% due 33 Basis'],
 'EMN': ['5 ¾% due 33 Mat. Matched CDS', '5 ¾% due 33 Basis'],
 'ENBCN': ['3 ⅛% due 29 Mat. Matched CDS', '3 ⅛% due 29 Basis'],
 'ENGIFP': ['5 ⅝% due 34 Mat. Matched CDS', '5 ⅝% due 34 Basis'],
 'ENIIM': ['5 ½% due 34 Mat. Matched CDS', '5 ½% due 34 Basis'],
 'EQNR': ['3 ⅛% due 30 Mat. Matched CDS', '3 ⅛% due 30 Basis'],
 'ET': ['4.9% due 35 Mat. Matched CDS', '4.9% due 35 Basis'],
 'EXC': ['5 ⅝% due 35 Mat. Matched CDS', '5 ⅝% due 35 Basis'],
 'EXPE': ['3 ¼% due 30 Mat. Matched CDS', '3 ¼% due 30 Basis'],
 'F': ['7.45% due 31 Mat. Matched CDS', '7.45% due 31 Basis'],
 'FCX': ['5.4% due 34 Mat. Matched CDS', '5.4% due 34 Basis'],
 'FDX': ['4 ¼% due 30 Mat. Matched CDS', '4 ¼% due 30 Basis'],
 'FE': ['4.55% due 30 Mat. Matched CDS', '4.55% due 30 Basis'],
 'GE': ['6 ¾% due 32 Mat. Matched CDS', '6 ¾% due 32 Basis'],
 'GIS': ['2 ⅞% due 30 Mat. Matched CDS', '2 ⅞% due 30 Basis'],
 'GM': ['5% due 35 Mat. Matched CDS', '5% due 35 Basis'],
 'HAL': ['2.92% due 30 Mat. Matched CDS', '2.92% due 30 Basis'],
 'HD': ['2.7% due 30 Mat. Matched CDS', '2.7% due 30 Basis'],
 'HES': ['7 ⅞% due 29 Mat. Matched CDS', '7 ⅞% due 29 Basis'],
 'KHC': ['6 ¾% due 32 Mat. Matched CDS', '6 ¾% due 32 Basis'],
 'HON': ['1.95% due 30 Mat. Matched CDS', '1.95% due 30 Basis'],
 'HPQ': ['3.4% due 30 Mat. Matched CDS', '3.4% due 30 Basis'],
 'HSBC': ['3.973% due 30 Mat. Matched CDS', '3.973% due 30 Basis'],
 'IBM': ['5 ⅞% due 32 Mat. Matched CDS', '5 ⅞% due 32 Basis'],
 'INTNED': ['2.727% due 32 Mat. Matched CDS', '2.727% due 32 Basis'],
 'INTC': ['4% due 32 Mat. Matched CDS', '4% due 32 Basis'],
 'JNJ': ['6.95% due 29 Mat. Matched CDS', '6.95% due 29 Basis'],
 'KMI': ['7.4% due 31 Mat. Matched CDS', '7.4% due 31 Basis'],
 'KPN': ['8 ⅜% due 30 Mat. Matched CDS', '8 ⅜% due 30 Basis'],
 'KR': ['7 ½% due 31 Mat. Matched CDS', '7 ½% due 31 Basis'],
 'L': ['6% due 35 Mat. Matched CDS', '6% due 35 Basis'],
 'LEN': ['5.2% due 30 Mat. Matched CDS', '5.2% due 30 Basis'],
 'LLOYDS': ['0% due 32 Mat. Matched CDS', '0% due 32 Basis'],
 'LMT': ['3.6% due 35 Mat. Matched CDS', '3.6% due 35 Basis'],
 'LNC': ['3.05% due 30 Mat. Matched CDS', '3.05% due 30 Basis'],
 'LOW': ['4 ½% due 30 Mat. Matched CDS', '4 ½% due 30 Basis'],
 'LUV': ['2 ⅝% due 30 Mat. Matched CDS', '2 ⅝% due 30 Basis'],
 'MCD': ['2 ⅝% due 29 Mat. Matched CDS', '2 ⅝% due 29 Basis'],
 'MCK': ['5.1% due 33 Mat. Matched CDS', '5.1% due 33 Basis'],
 'MDC': ['3.85% due 30 Mat. Matched CDS', '3.85% due 30 Basis'],
 'MDLZ': ['2 ¾% due 30 Mat. Matched CDS', '2 ¾% due 30 Basis'],
 'MET': ['6 ½% due 32 Mat. Matched CDS', '6 ½% due 32 Basis'],
 'MO': ['3.4% due 30 Mat. Matched CDS', '3.4% due 30 Basis'],
 'MPC': ['5.15% due 30 Mat. Matched CDS', '5.15% due 30 Basis'],
 'MSI': ['2.3% due 30 Mat. Matched CDS', '2.3% due 30 Basis'],
 'MTNA': ['6.8% due 32 Mat. Matched CDS', '6.8% due 32 Basis'],
 'NEM': ['5 ⅞% due 35 Mat. Matched CDS', '5 ⅞% due 35 Basis'],
 'NFLX': ['5 ⅜% due 29 Mat. Matched CDS', '5 ⅜% due 29 Basis'],
 'NGGLN': ['5.809% due 33 Mat. Matched CDS', '5.809% due 33 Basis'],
 'NOC': ['4.4% due 30 Mat. Matched CDS', '4.4% due 30 Basis'],
 'NSC': ['2.55% due 29 Mat. Matched CDS', '2.55% due 29 Basis'],
 'NWG': ['5.076% due 30 Mat. Matched CDS', '5.076% due 30 Basis'],
 'OMC': ['2.45% due 30 Mat. Matched CDS', '2.45% due 30 Basis'],
 'ORAFP': ['8 ½% due 31 Mat. Matched CDS', '8 ½% due 31 Basis'],
 'ORCL': ['4.3% due 34 Mat. Matched CDS', '4.3% due 34 Basis'],
 'OVV': ['8 ⅛% due 30 Mat. Matched CDS', '8 ⅛% due 30 Basis'],
 'OXY': ['7 ½% due 31 Mat. Matched CDS', '7 ½% due 31 Basis'],
 'PARA': ['7 ⅞% due 30 Mat. Matched CDS', '7 ⅞% due 30 Basis'],
 'PFE': ['6 ½% due 34 Mat. Matched CDS', '6 ½% due 34 Basis'],
 'PG': ['5.8% due 34 Mat. Matched CDS', '5.8% due 34 Basis'],
 'PHM': ['7 ⅞% due 32 Mat. Matched CDS', '7 ⅞% due 32 Basis'],
 'PKG': ['3% due 29 Mat. Matched CDS', '3% due 29 Basis'],
 'PRU': ['5 ¾% due 33 Mat. Matched CDS', '5 ¾% due 33 Basis'],
 'R': ['6.6% due 33 Mat. Matched CDS', '6.6% due 33 Basis'],
 'RIFP': ['1 ⅝% due 31 Mat. Matched CDS', '1 ⅝% due 31 Basis'],
 'RTX': ['7 ½% due 29 Mat. Matched CDS', '7 ½% due 29 Basis'],
 'SHW': ['2.3% due 30 Mat. Matched CDS', '2.3% due 30 Basis'],
 'SO': ['2.65% due 29 Mat. Matched CDS', '2.65% due 29 Basis'],
 'SRE': ['5 ½% due 33 Mat. Matched CDS', '5 ½% due 33 Basis'],
 'STANLN': ['4.305% due 30 Mat. Matched CDS', '4.305% due 30 Basis'],
 'T': ['4 ½% due 35 Mat. Matched CDS', '4 ½% due 35 Basis'],
 'TGT': ['6.35% due 32 Mat. Matched CDS', '6.35% due 32 Basis'],
 'TSN': ['4 ⅞% due 34 Mat. Matched CDS', '4 ⅞% due 34 Basis'],
 'UBS': ['3.126% due 30 Mat. Matched CDS', '3.126% due 30 Basis'],
 'UNH': ['4 ⅝% due 35 Mat. Matched CDS', '4 ⅝% due 35 Basis'],
 'UNP': ['3 ⅜% due 35 Mat. Matched CDS', '3 ⅜% due 35 Basis'],
 'VLO': ['7 ½% due 32 Mat. Matched CDS', '7 ½% due 32 Basis'],
 'VZ': ['7 ¾% due 30 Mat. Matched CDS', '7 ¾% due 30 Basis'],
 'WHR': ['2.4% due 31 Mat. Matched CDS', '2.4% due 31 Basis'],
 'WMB': ['7 ½% due 31 Mat. Matched CDS', '7 ½% due 31 Basis'],
 'WMT': ['7.55% due 30 Mat. Matched CDS', '7.55% due 30 Basis'],
 'WY': ['7 ⅜% due 32 Mat. Matched CDS', '7 ⅜% due 32 Basis']}

In [14]:
#################################### Changing order of names

issuer = ['All'] + list(sorted(set([item.split(' ',1)[0] for item in options])))
options1 = list(set([item.split(' ',1)[1] for item in options])) + ['US 5Y','US 7Y','US 10Y']

df1 = pd.DataFrame()

dropdown1 = Dropdown(options=issuer, description='Issuer')
dropdown2 = Dropdown(options=options1, description='A:')
dropdown3 = Dropdown(options=options1, description='B:')
dropdown4 = Dropdown(options=options1, description='C:')
dropdown5 = Dropdown(options=options1, description='D:')
dropdown6 = Dropdown(options=options1, description='E:')
dropdown7 = Dropdown(options=options1, description='F:')
dropdown8 = Dropdown(options=options1, description='G:')
dropdown9 = Dropdown(options=options1, description='H:')
dropdown10 = Dropdown(options=options1, description='I:')
dropdown11 = Dropdown(options=options1, description='J:')
dropdown12 = Dropdown(options=options1, description='K:')
dropdown13 = Dropdown(options=options1, description='L:')
# dropdown14 = Dropdown(options=time, description='Time:')

dropdown14 = widgets.FloatText(
    value=5,
    description='Years:',
    step=0.01
)

row1 = HBox([dropdown1])
row2 = HBox([dropdown2, dropdown3, dropdown4])
row3 = HBox([dropdown5, dropdown6, dropdown7])
row4 = HBox([dropdown8, dropdown9, dropdown10])
row5 = HBox([dropdown11, dropdown12, dropdown13])
row6 = HBox([dropdown14])

dropdown_box = VBox([row1, row2, row3, row4, row5, row6])

display(dropdown_box)

expression_input1 = Text(description='Horizontal Axis:',value='a')
expression_input2 = Text(description='Vertical Axis:',value='b')
display(expression_input1)
display(expression_input2)

def get_values():
    return dropdown1.value, dropdown2.value, dropdown3.value, \
    dropdown4.value, dropdown5.value, dropdown6.value, \
    dropdown7.value, dropdown8.value, dropdown9.value, \
    dropdown10.value, dropdown11.value, dropdown12.value, \
    dropdown13.value, dropdown14.value

submit_button = Button(description="Submit")
display(submit_button)

output = Output()
display(output)

def on_button_clicked(b):
    with output:
        global df
        clear_output()
        selected_values = get_values()
        x = list(selected_values)
        if x[0] != 'All':
            x = [x[0]] + [f"{x[0]} {item}" for item in x[1:-1]] + [x[-1]]
            selected_values = [x[0]] + \
            ['BBG ' + item.split(' ',1)[1] if (len(item.split(' '))>2 and item.split(' ')[1]=='US') else item for item in x[1:-1]]\
            +[x[-1]]
            selected_values = tuple(selected_values)
        
        expression1 = expression_input1.value.upper()
        expression2 = expression_input2.value.upper()
        
        col_map = {'A': selected_values[1], 'B': selected_values[2], 'C': selected_values[3], 
                   'D': selected_values[4], 'E': selected_values[5], 'F': selected_values[6],
                   'G': selected_values[7], 'H': selected_values[8], 'I': selected_values[9], 
                   'J': selected_values[10], 'K': selected_values[11], 'L': selected_values[12]}
        
        for key, value in col_map.items():
            expression1 = expression1.replace(key, key.lower())
            expression2 = expression2.replace(key, key.lower())
        
        a, b, c, d, e, f, g, h, i, j, k, l = sp.symbols('a b c d e f g h i j k l')
        expr1 = sp.sympify(expression1)
        expr11 = sp.sympify(expression1)
        expr2 = sp.sympify(expression2) if expression2 else None
        expr21 = sp.sympify(expression2) if expression2 else None

        cols_to_check = [col_map['A'], col_map['B'], col_map['C'], \
                         col_map['D'], col_map['E'], col_map['F'], \
                         col_map['G'], col_map['H'], col_map['I'], \
                         col_map['J'], col_map['K'], col_map['L'] ]
        
        df1 = df.copy()
        df2 = df.copy()
        
        replacements = {
            'A': selected_values[1], 'a': selected_values[1],
            'B': selected_values[2], 'b': selected_values[2],
            'C': selected_values[3], 'c': selected_values[3],
            'D': selected_values[4], 'd': selected_values[4],
            'E': selected_values[5], 'e': selected_values[5],
            'F': selected_values[6], 'f': selected_values[6],
            'G': selected_values[7], 'g': selected_values[7],
            'H': selected_values[8], 'h': selected_values[8],
            'I': selected_values[9], 'i': selected_values[9],
            'J': selected_values[10], 'j': selected_values[10],
            'K': selected_values[11], 'k': selected_values[11],
            'L': selected_values[12], 'l': selected_values[12]
        }

        regex = re.compile("|".join(re.escape(key) for key in replacements.keys()))
        used_keys = set()
        def substitution(match):
            key = match.group(0)
            used_keys.add(key)
            return replacements[key]
        
        expr111 = regex.sub(substitution, str(expr11))
        expr211 = regex.sub(substitution, str(expr21))
        used_substitutions = list(used_keys)

        used_substitutions = [replacements[item.upper()] for item in used_substitutions]

        df1['Expression 1'] = df1.apply(lambda row: expr1.evalf(subs={\
            a: row[col_map['A']], b: row[col_map['B']], 
            c: row[col_map['C']], d: row[col_map['D']], 
            e: row[col_map['E']], f: row[col_map['F']],
            g: row[col_map['G']], h: row[col_map['H']],
            i: row[col_map['I']], j: row[col_map['J']], 
            k: row[col_map['K']], l: row[col_map['L']] }), axis=1).astype(float)       
        df1['Expression 2'] = df1.apply(lambda row: expr2.evalf(subs={\
            a: row[col_map['A']], b: row[col_map['B']], 
            c: row[col_map['C']], d: row[col_map['D']], 
            e: row[col_map['E']], f: row[col_map['F']],
            g: row[col_map['G']], h: row[col_map['H']],
            i: row[col_map['I']], j: row[col_map['J']], 
            k: row[col_map['K']], l: row[col_map['L']] }) if expr2 else np.nan, axis=1).astype(float)

        plot_exp1 = np.nan
        plot_exp2 = np.nan
        
        if ('FWD' in expr111) and not ('FWD' in expr211) and expression2:
            df2['Expression 2'] = df2.apply(lambda row: expr2.evalf(subs={\
            a: row[col_map['A']], b: row[col_map['B']], 
            c: row[col_map['C']], d: row[col_map['D']], 
            e: row[col_map['E']], f: row[col_map['F']],
            g: row[col_map['G']], h: row[col_map['H']],
            i: row[col_map['I']], j: row[col_map['J']], 
            k: row[col_map['K']], l: row[col_map['L']] }), axis=1).astype(float)
            plot_exp2 = df2.iloc[-1]['Expression 2'].astype(float)
        if ('FWD' in expr211) and not ('FWD' in expr111):
            df2['Expression 1'] = df2.apply(lambda row: expr1.evalf(subs={\
            a: row[col_map['A']], b: row[col_map['B']], 
            c: row[col_map['C']], d: row[col_map['D']], 
            e: row[col_map['E']], f: row[col_map['F']],
            g: row[col_map['G']], h: row[col_map['H']],
            i: row[col_map['I']], j: row[col_map['J']], 
            k: row[col_map['K']], l: row[col_map['L']] }), axis=1).astype(float)
            plot_exp1 = df2.iloc[-1]['Expression 1'].astype(float)

        
        def filter_df(dfx1, time_period):
            dfx = dfx1.copy()
            dfx.index=pd.to_datetime(dfx.index)
            if time_period == 'All':
                return dfx
            else:
                now = datetime.now()
                # if time_period == '2Y':
                #     start_date = now - timedelta(days=2*365)
                # elif time_period == '1Y':
                #     start_date = now - timedelta(days=365)
                # elif time_period == '6M':
                #     start_date = now - timedelta(days=6*30)
                # elif time_period == '3M':
                #     start_date = now - timedelta(days=3*30)
                # return dfx[dfx.index.date >= start_date.date()]   
                
                start_date = now - timedelta(days=int(time_period*365))
                return dfx[dfx.index.date >= start_date.date()] 

        def update_plot(plot_exp1,plot_exp2,df1,df2):
            df1 = df1[used_substitutions + ['Expression 1','Expression 2']].dropna().copy()
            # display(df1)
            if expression2:
                fig = make_subplots(rows=2, cols=1, shared_xaxes=False, \
                                    vertical_spacing=0.1, specs=[ [{"secondary_y":False}], [{"secondary_y":True}] ],\
                                   subplot_titles=(f'Scatter Plot from {df1.index[0].date()} to {df1.index[-1].date()}',\
                                                  f'{expr111}: {df1[df1.columns[-2]][-1]:.4f}   &   {expr211}: {df1[df1.columns[-1]][-1]:.4f}'))
                
                x_scatter = df1[df1.columns[-2]]
                y_scatter = df1[df1.columns[-1]]

                coeffs = np.polyfit(x_scatter,y_scatter,2)
                polynomial = np.poly1d(coeffs)
                x_poly=np.linspace(min(x_scatter),max(x_scatter),500)
                y_poly=polynomial(x_poly)
                y_pred = polynomial(x_scatter)
                r2=r2_score(y_scatter,y_pred)
                equation_text = f"y = {coeffs[0]:.4f}*x² + {coeffs[1]:.4f}*x + {coeffs[2]:.4f}"
    
                residuals = y_scatter-y_pred
                res_sum=np.sum(residuals**2)
                dof=len(x_scatter)-len(coeffs)
                res_var = res_sum / dof
                se = np.sqrt(res_var)
                conf_upper = y_poly + 2 * se
                conf_lower = y_poly - 2 * se
                
                fig.add_trace(go.Scatter(x=df1[df1.columns[-2]][-63:-1], y=df1[df1.columns[-1]][-63:-1], \
                                         name='< 3 months', mode='markers',marker=dict(color='green')), \
                              row=1,col=1,secondary_y=False)
                fig.add_trace(go.Scatter(x=df1[df1.columns[-2]][-252:-63], y=df1[df1.columns[-1]][-252:-63], \
                                         name='> 3 months & < 1 year', mode='markers',marker=dict(color='blue')), \
                              row=1,col=1,secondary_y=False)
                fig.add_trace(go.Scatter(x=df1[df1.columns[-2]][:-252], y=df1[df1.columns[-1]][:-252], \
                                         name='> 1 year', mode='markers',marker=dict(color='gray')), \
                              row=1,col=1,secondary_y=False)
                
                fig.add_trace(go.Scatter(x=[df1[df1.columns[-2]].iloc[-1]], y=[df1[df1.columns[-1]].iloc[-1]], \
                                         name=f'Value as of {df1.index[-1].date()}', mode='markers',marker=dict(color='red',size=10),\
                                         showlegend=True), row=1,col=1,secondary_y=False)      
                fig.add_trace(go.Scatter(x=x_poly, y=y_poly, \
                                         name=f'Polynomial Fit<br>R² = {r2:.3f}<br>{equation_text}', \
                                         mode='lines', line=dict(color='orange'),
                                         showlegend=True), row=1,col=1,secondary_y=False) 
               
                fig.add_trace(go.Scatter(x=x_poly, y=conf_upper, \
                                         name='Upper Confidence Band', \
                                         mode='lines', line=dict(color='orange',dash='dash'),
                                         showlegend=True), row=1,col=1,secondary_y=False) 
                fig.add_trace(go.Scatter(x=x_poly, y=conf_lower, \
                                         name='Lower Confidence Band', fill='tonexty',\
                                         fillcolor='rgba(255,165,0,0.15)',
                                         mode='lines', line=dict(color='orange',dash='dash'),
                                         showlegend=True), row=1,col=1,secondary_y=False) 
                
                if ('FWD' in expr111) and not ('FWD' in expr211):
                    # display(f"Condition1")
                    plot_exp2 = [plot_exp2] * len(df1[df1.columns[-2]])
                    fig.add_trace(go.Scatter(x=df1[df1.columns[-2]], y=plot_exp2, mode='lines',\
                     line=dict(color='red',width=1),name = f'{expr211} on {df2.index[-1]} was {plot_exp2[0]}'), \
                      row=1,col=1,secondary_y=False)

                if ('FWD' in expr211) and not ('FWD' in expr111):
                    # display(f"Condition2")
                    plot_exp1 = [plot_exp1] * len(df1[df1.columns[-1]])
                    fig.add_trace(go.Scatter(x=plot_exp1, y=df1[df1.columns[-1]], mode='lines',\
                     line=dict(color='red',width=1), name = f'{expr111} on {df2.index[-1]} was {plot_exp1[0]}'), \
                      row=1,col=1,secondary_y=False)
                
                fig.add_trace(go.Scatter(x=df1.index, y=df1[df1.columns[-2]], name=expr111,\
                         line=dict(color='blue')), row=2,col=1,secondary_y=False)
                
                fig.add_trace(go.Scatter(x=[df1.index[0], df1.index[-1]], \
                 y=[df1[df1.columns[-2]][-1], df1[df1.columns[-2]][-1]], \
                 mode='lines', line=dict(dash='dash', color='blue'), \
                 name=f'Recent value of {expr111} is {df1[df1.columns[-2]][-1]}'),row=2,col=1,secondary_y=False)
                
                fig.add_trace(go.Scatter(x=df1.index, y=df1[df1.columns[-1]], \
                                     name=expr211, line=dict(color='green')),\
                                      row=2,col=1, secondary_y=True)
                
                fig.add_trace(go.Scatter(x=[df1.index[0], df1.index[-1]], \
                 y=[df1[df1.columns[-1]][-1], df1[df1.columns[-1]][-1]], \
                 mode='lines', line=dict(dash='dash', color='green'), \
                 name=f'Recent value of {expr211} is {df1[df1.columns[-1]][-1]}'),row=2,col=1, secondary_y=True)
                
                fig.update_layout(
                hovermode='x unified',
                plot_bgcolor='white',
                paper_bgcolor='white',
                legend=dict(orientation="h",yanchor="bottom",xanchor="center",x=0.5,y=1.05),
                height=1900,
                width=1100)

            else:
                fig = make_subplots(rows=1, cols=1, shared_xaxes=False, \
                                    vertical_spacing=0.1, subplot_titles=(f'{expr111}: {df1[df1.columns[-2]][-1]:.4f}'))
                
                fig.add_trace(go.Scatter(x=df1.index, y=df1[df1.columns[-2]], name=expr111,\
                         line=dict(color='blue')), row=1,col=1)
                
                fig.add_trace(go.Scatter(x=[df1.index[0], df1.index[-1]], \
                 y=[df1[df1.columns[-2]][-1], df1[df1.columns[-2]][-1]], \
                 mode='lines', line=dict(dash='dash', color='blue'), \
                 name=f'Recent value of {expr111} is {df1[df1.columns[-2]][-1]}'))

                fig.update_layout(
                yaxis=dict(zeroline=True, zerolinecolor='black'),
                hovermode='x unified',
                plot_bgcolor='white',
                paper_bgcolor='white',
                legend=dict(orientation="h",yanchor="bottom",xanchor="center",x=0.5,y=1.05),
                height=800,
                width=1100)

            def split_label(label,n):
                return '<br>'.join([label[i:i+n] for i in range(0,len(label),n)])

            label1 = split_label(expr111,38)
            label2 = split_label(expr211,38)
          
            if expression2:
                fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True,\
                                 row=1,col=1,title=label1, showgrid=True, gridcolor='LightGrey',gridwidth=1)
                fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True,\
                                 row=1,col=1, title=label2, secondary_y=False, showgrid=True, gridcolor='LightGrey',gridwidth=1)
                fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True, \
                                 title='Date', row=2, col=1, showgrid=True, gridcolor='LightGrey',gridwidth=1)
                fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True, \
                                 title=label1, side='left', row=2,col=1, secondary_y=False, \
                                 showgrid=True, gridcolor='LightGrey',gridwidth=1, title_font = dict(color='blue'))
                fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True, \
                                 title=label2, side='right', row=2,col=1, secondary_y=True, title_font = dict(color='green'))
            else:
                fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True, \
                                 title='Date', row=1, col=1, showgrid=True, gridcolor='LightGrey',gridwidth=1)
                fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True, \
                                 title=label1, side='left', row=1,col=1, secondary_y=False, showgrid=True, gridcolor='LightGrey',gridwidth=1)
            fig.show()
        
        df1 = filter_df(df1, selected_values[-1]).copy() #Time goes here
        interact(update_plot(plot_exp1,plot_exp2,df1,df2))

submit_button.on_click(on_button_clicked)

def update_dropdowns(*args):
    selected_issuer = dropdown1.value
    if selected_issuer:
        for i in range(2, 14):
            dropdown = globals().get(f'dropdown{i}')
            if dropdown:
                if selected_issuer != 'All':
                    x = list([item.split(' ', 1)[1] for item in options if item.startswith(selected_issuer)]) + ['US 5Y','US 7Y','US 10Y']
                else:
                    x = options
                dropdown.options = x
        if selected_issuer in list(default_values_dict.keys()):
            globals().get(f'dropdown2').value = default_values_dict[selected_issuer][0]
            globals().get(f'dropdown3').value = default_values_dict[selected_issuer][1]

dropdown1.observe(update_dropdowns, names='value')
update_dropdowns()

Text(value='a', description='Horizontal Axis:')

Text(value='b', description='Vertical Axis:')

Button(description='Submit', style=ButtonStyle())

Output()